# Setup

In [58]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
from pathlib import Path
import torch
import matplotlib.pyplot as plt

In [60]:
# data_path = Path('/tmp/am_segm/inference/Crops of Exp9_FD')
# data_path = Path('/opt/data/am_segm/inference/Crops of FDtest Exp8A')
data_path = Path('/opt/data/am_segm/inference/Jul19 test')

In [61]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

# Predict Masks

In [66]:
import segmentation_models_pytorch as smp

from am_segm.preprocess import convert_to_tiles, stitch_tiles_at_path
from am_segm.dataset import load_ds, valid_transform
from am_segm.utils import predict_save, overlay_images_with_masks

In [52]:
convert_to_tiles(data_path / 'source')

Converting images to tiles
Splitting /opt/data/am_segm/inference/Jul19 test/source/Vitaly-test-AM_cropped/source.tiff
Already exists: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/000.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/001.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/002.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/003.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/004.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/005.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/006.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM

Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/088.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/089.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/090.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/091.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/092.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/093.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/094.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/095.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/096.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/sourc

Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/181.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/182.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/183.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/184.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/185.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/186.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/187.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/188.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/source/189.png
Save tile: /opt/data/am_segm/inference/Jul19 test/source_tiles/Vitaly-test-AM_cropped/sourc

In [53]:
model = smp.Unet(encoder_name='se_resnext50_32x4d', encoder_weights=None, decoder_use_batchnorm=True)
model.load_state_dict(torch.load('unet.pt', map_location=device))
model.to(device);

In [54]:
ds = load_ds(data_path / 'source_tiles', valid_transform())
len(ds)

225

In [55]:
%time predict_save(model, ds, data_path / 'pseudo_tiles')

Vitaly-test-AM_cropped
Vitaly-test-AM_cropped
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/000.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/000.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/001.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/001.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/002.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/002.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/003.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/003.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/004.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/004.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-

/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/073.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/074.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/074.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/075.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/075.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/076.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/076.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/077.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/077.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/078.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/078.png
/opt/data/am_segm/infe

/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/145.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/146.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/146.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/147.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/147.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/148.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/148.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/149.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/149.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/150.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/150.png
/opt/data/am_segm/infe

/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/216.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/216.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/217.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/217.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/218.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/218.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/219.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/219.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/220.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/mask/220.png
/opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped/source/221.png
/opt/data/am_segm/in

In [64]:
stitch_tiles_at_path(data_path / 'pseudo_tiles', data_path / 'source_tiles',
                     overwrite=True, image_ext='tiff')

Stitching tiles at /opt/data/am_segm/inference/Jul19 test/pseudo_tiles/Vitaly-test-AM_cropped
Saved stitched image to /opt/data/am_segm/inference/Jul19 test/pseudo_tiles_stitched/Vitaly-test-AM_cropped/source.tiff
Saved stitched image to /opt/data/am_segm/inference/Jul19 test/pseudo_tiles_stitched/Vitaly-test-AM_cropped/mask.tiff


In [68]:
overlay_images_with_masks(data_path / 'pseudo_tiles_stitched', image_ext='tiff')

Overlaying: /opt/data/am_segm/inference/Jul19 test/pseudo_tiles_stitched/Vitaly-test-AM_cropped


---